**Name:** Yidu WU

**EID:** yiduwu2

**Kaggle Team Name:** Yidu WU

# CS5489 - Assignment 1 - Tweet Sentiment Analysis

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [22]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
random.seed(100)

In [23]:
def read_text_data(fname):
    txtdata = []
    classes = []
    topics  = []
    with open(fname, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # get the text
            txtdata.append(row[0])
            # get the class (convert to integer)
            if len(row)>1:
                classes.append(row[1])
                topics.append(row[2])
    
    if (len(classes)>0) and (len(txtdata) != len(classes)):        
        raise Exception("mismatched length!")
    
    return (txtdata, classes, topics)

def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    tmp = [['Id', 'Prediction']]
    
    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)
        
    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

In [24]:
# load the data
# (if using Kaggle notebooks you need to include the directory path: /kaggle/input/cs5489-2020b-assignment-1/)
(traintxt, trainY, traintopic) = read_text_data("sanders_tweets_train.txt")
(testtxt, _, _)                = read_text_data("sanders_tweets_test.txt")

print(len(traintxt))
print(len(testtxt))

2396
1028


In [25]:
# Deal with train data
# Delete unrelated words by the Regular expression
import re
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)    
    return input_txt  

def replace_all_blank(value):
    result = re.sub('\W+', ' ', value).replace("_", '')
    return result

#delete some special words
print(traintxt[0:20])
print("")
print("process step1")
traintxtTemp = []
for i in range(len(traintxt)):
    t = remove_pattern(traintxt[i], "#[\w]*")            #Delete #
    t = remove_pattern(t, "@[\w]*")                      #Delete @ 
    t = remove_pattern(t, r'http://[a-zA-Z0-9.?/&=:]*')  #Delete Url
    t = replace_all_blank(t)                             
    t = remove_pattern(t, r"[0-9]+")                     #Delete number
    traintxtTemp.append(t)
print(traintxtTemp[0:20])
print("")
print("process step2")

#Delete words which length less than 4
traintxtTemp2 = []
f = lambda x: ' '.join([w for w in x.split() if len(w)>3])
traintxtTemp2 = list(map(f, traintxtTemp))
print(traintxtTemp2[0:20])

['http://t.co/QV4m1Un9 Forget the phone.. Nice UI. Liking the Scroll Feature #android #google #nexus', 'I hate when my phne do what it want on #twitter', '…and only the first 23 images in my Photo Roll made it over? Seriously, @apple, how did you fuck this up so much?', 'The lock screen now has facial recognition capability! #Google #Android #ICS', '#TeamGoogleNexus RT @B__Y #Google + #Samsung = Perfect #Icecream sandwich #GalaxyNexus', "RT @FierceWireless: #Microsoft's Ballmer promises #Nokia Windows phones next week, slams #Android.  http://t.co/OH8Btu2e #WP7 #Window...", '@Slater_Boy Needs to take his ass to sleep... Its grown folks hour on #twitter', 'RT @bmann Awesome! @TommyLee is moving to #Vancouver as Dev Evangelist for #Microsoft - we finally get a local', 'Have never had such poor customer service at @Apple before! What happened? (@ Apple Store w/ 2 others) http://t.co/GKlXMUi6', 'Thank you @apple for your innovations. Exhibit A: Guy playing with Facetime instead of watching

In [ ]:
# install nltk to ues the Stemming function
pip install nltk

In [26]:
# Deal with train data
# use Stemming to extract the stem
import nltk
from nltk.stem.porter import PorterStemmer  
porter_stemmer = PorterStemmer()

print(traintxtTemp2[0:20])
print("")
print("process step3")
traintxtTemp3 = []
f = lambda x: x.split()
traintxtTemp3 = list(map(f, traintxtTemp2))
print(traintxtTemp3[0:20])
print("")

print("process step4")
traintxtNew = []
f=lambda x: [porter_stemmer.stem(i) for i in x]
traintxtNew = list(map(f, traintxtTemp3))
print(traintxtNew[0:20])

['Forget phone Nice Liking Scroll Feature', 'hate when phne what want', 'only first images Photo Roll made over Seriously fuck this much', 'lock screen facial recognition capability', 'Perfect sandwich', 'Ballmer promises Windows phones next week slams', 'Needs take sleep grown folks hour', 'Awesome moving Evangelist finally local', 'Have never such poor customer service before What happened Apple Store others', 'Thank your innovations Exhibit playing with Facetime instead watching game sports', 'cannot wait phone', 'Child View', 'Ways Business', 'Google updates Nexus site with Galaxy Nexus details', 'Thank giving tweets that have been tweeted past days', 'Funny Google ranked', 'iPhone camera SONY', 'Yahoo Profit Drops Cent', 'Soon Getting ready party', 'need unfollow some these wannabe real thugs have Hell']

process step3
[['Forget', 'phone', 'Nice', 'Liking', 'Scroll', 'Feature'], ['hate', 'when', 'phne', 'what', 'want'], ['only', 'first', 'images', 'Photo', 'Roll', 'made', 'over', 

In [ ]:
# install mosestokenizer to combine the words which have been stemmed
pip install mosestokenizer

In [27]:
# Deal with train data
# use mosestokenizer to combine the words
from mosestokenizer import MosesDetokenizer
detokenizer = MosesDetokenizer()
for i in range(len(traintxtNew)):
    traintxtNew[i] = detokenizer(traintxtNew[i])
    
print("process step5")
print(traintxtNew[0:20])

process step5
['forget phone nice like scroll featur', 'hate when phne what want', 'onli first imag photo roll made over serious fuck thi much', 'lock screen facial recognit capabl', 'perfect sandwich', 'ballmer promis window phone next week slam', 'need take sleep grown folk hour', 'awesom move evangelist final local', 'have never such poor custom servic befor what happen appl store other', 'thank your innov exhibit play with facetim instead watch game sport', 'cannot wait phone', 'child view', 'way busi', 'googl updat nexu site with galaxi nexu detail', 'thank give tweet that have been tweet past day', 'funni googl rank', 'iphon camera soni', 'yahoo profit drop cent', 'soon get readi parti', 'need unfollow some these wannab real thug have hell']


In [28]:
# Deal with test data
# Delete unrelated words by the Regular expression
print(testtxt[0:20])
print("")
print("process step1")
testtxtTemp = []
for i in range(len(testtxt)):
    t = remove_pattern(testtxt[i], "#[\w]*")            #Delete #
    t = remove_pattern(t, "@[\w]*")                      #Delete @ 
    t = remove_pattern(t, r'http://[a-zA-Z0-9.?/&=:]*')  #Delete Url
    t = replace_all_blank(t)                             
    t = remove_pattern(t, r"[0-9]+")                     #Delete number
    testtxtTemp.append(t)
print(testtxtTemp[0:20])
print("")
print("process step2")

#Delete words which length less than 4
testtxtTemp2 = []
f = lambda x: ' '.join([w for w in x.split() if len(w)>3])
testtxtTemp2 = list(map(f, testtxtTemp))
print(testtxtTemp2[0:20])

# use Stemming to extract the stem
print("")
print("process step3")
testtxtTemp3 = []
f = lambda x: x.split()
testtxtTemp3 = list(map(f, testtxtTemp2))
print(testtxtTemp3[0:20])
print("")

print("process step4")
testtxtNew = []
f=lambda x: [porter_stemmer.stem(i) for i in x]
testtxtNew = list(map(f, testtxtTemp3))
print(testtxtNew[0:20])

# use mosestokenizer to combine the words
detokenizer_test = MosesDetokenizer()
for i in range(len(testtxtNew)):
    testtxtNew[i] = detokenizer_test(testtxtNew[i])

print("")
print("process step5")
print(testtxtNew[0:20])

['What do we have to do to get Flash on #iPad? @Apple please please please change your mind. Thank you in advance! @kusi_sk  @PeterNemcok', 'RT @RUILIFESTYLE: #GOOGLE THE MIXTAPE http://t.co/v8zTtNVV', 'Apples to oranges maybe, but #Twitter is way more engaging than #FB. Content vs. Stalkerismo', 'Thanks @apple for an OS update that has only succesfully bricked my iPod.', 'WHTS GOIN ON #TWITTER FAM', 'Awesome service from the @apple store in pc. Thanks chris!', 'All this talk about about how good #icecreamsandwich looks is making me hungry #google #omnomnomnom', "RT @EcommNewsUpdate Steve Yegge's Google Plus Rant - Google's Pathetic Afterthought http://t.co/4Tt5OsvN #blog #facebook #google #googleplus", "Comp crash #Microsoft :/ Wouldn't get this with an iMac -__-", '@Affan Fact! They still do everything better though #google', '#Microsoft Research shows #Holodesk - looks like science fiction, but is science fact http://t.co/vv6ltkYp', 'Microsoft applies for celebrity search patent #mi

In [29]:
# Use the SVM with RBF kernel
cntvect = feature_extraction.text.CountVectorizer(stop_words='english')
cntvect.fit(traintxtNew)
trainX = cntvect.transform(traintxtNew)
testX  = cntvect.transform(testtxtNew)
paramgrid = {'C': logspace(-2,3,20), 'gamma': logspace(-4,3,20) }
svmcv = model_selection.GridSearchCV(svm.SVC(kernel='rbf'), paramgrid, cv=10,
                                    n_jobs=-1, verbose=True)
svmcv.fit(trainX, trainY);
predY = svmcv.predict(testX)
print(predY)
write_csv_kaggle_sub("my_submission.csv", predY)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   51.3s
[Parallel(n_jobs=-1)]: Done 704 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1154 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1704 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 2354 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 3104 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 4000 out of 4000 | elapsed:  8.5min finished


['neutral' 'neutral' 'neutral' ... 'neutral' 'neutral' 'neutral']
